In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST

from sklearn import preprocessing

import os
import gc
import sys

In [65]:
d = pd.read_csv('peqw.csv')

In [66]:
d.rename(index=str, columns = {'Unnamed: 0':'time'}, inplace=True)

In [67]:
d.head()

,time,year,absacc,acc,age,agr,bm,bm_ia,cash,cashdebt,...,ps,nincr,divi,divo,rd,sin,IPO,convind,securedind,month
0,0,1980,0.020437,-0.019961,0.004841,-0.051247,0.002593,0.026180,0.005222,-0.066361,...,-0.021583,0.003167,-0.031233,-0.031233,-0.031233,0.096761,0.096761,0.004620,0.096761,1
1,1,1980,0.015636,-0.000363,-0.018122,-0.014145,-0.021253,-0.004250,0.003154,-0.027831,...,-0.023228,-0.017170,-0.051512,-0.051512,-0.051512,-0.014712,-0.014712,-0.003763,-0.014712,2
2,2,1980,-0.040065,-0.018086,-0.014526,0.001264,0.033694,0.022895,-0.003068,0.042815,...,0.025033,0.008833,0.011847,0.011847,0.011847,-0.160399,-0.160399,-0.008245,-0.160399,3
3,3,1980,-0.017147,-0.013814,0.001677,0.001489,-0.022916,-0.014910,-0.015351,0.007168,...,0.027996,0.029631,0.035142,0.035142,0.035142,0.053434,0.053434,0.000111,0.053434,4
4,4,1980,-0.005577,0.005240,-0.004074,0.029255,-0.010575,-0.011134,0.015279,0.022806,...,0.004142,-0.003777,0.019834,0.019834,0.019834,0.073763,0.073763,0.006476,0.073763,5


In [68]:
d.shape

(420, 105)

In [69]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [70]:
scaler.fit(d.drop('mom1m', axis=1))

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [71]:
d.shape

(420, 105)

In [72]:
scaled_feat = scaler.transform(d.drop('mom1m', axis=1))
d_feat = pd.DataFrame(scaled_feat, columns=list(d.columns).remove('mom1m'))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [73]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled_feat,
                                                    d['mom1m'],
                                                    test_size=0.30)

In [74]:
X_train = X_train.astype(np.float32)
T_train = torch.from_numpy(X_train)

In [75]:
y_train = y_train.values
p_train = y_train.astype(np.float32)
t_train = torch.from_numpy(p_train)

In [76]:
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(104, 1)
        
    def forward(self, x):
        out = self.linear(x)
        return out


In [77]:
model = LinearRegression()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=1e-4)

In [90]:
num_epochs = 1000
for epoch in range(num_epochs):
    inputs = Variable(T_train)
    target = Variable(t_train)

    # forward
    out = model(inputs)
    loss = criterion(out, target)
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 20 == 0:
        print('Epoch[{}/{}], loss: {:.6f}'
              .format(epoch+1, num_epochs, loss.item()))

Epoch[20/1000], loss: 0.242826
Epoch[40/1000], loss: 0.230587
Epoch[60/1000], loss: 0.219616
Epoch[80/1000], loss: 0.209760
Epoch[100/1000], loss: 0.200886
Epoch[120/1000], loss: 0.192878
Epoch[140/1000], loss: 0.185635
Epoch[160/1000], loss: 0.179066
Epoch[180/1000], loss: 0.173094
Epoch[200/1000], loss: 0.167649
Epoch[220/1000], loss: 0.162672
Epoch[240/1000], loss: 0.158110
Epoch[260/1000], loss: 0.153915
Epoch[280/1000], loss: 0.150047
Epoch[300/1000], loss: 0.146471
Epoch[320/1000], loss: 0.143154
Epoch[340/1000], loss: 0.140069
Epoch[360/1000], loss: 0.137191
Epoch[380/1000], loss: 0.134499
Epoch[400/1000], loss: 0.131973
Epoch[420/1000], loss: 0.129598
Epoch[440/1000], loss: 0.127357
Epoch[460/1000], loss: 0.125239
Epoch[480/1000], loss: 0.123231
Epoch[500/1000], loss: 0.121323
Epoch[520/1000], loss: 0.119506
Epoch[540/1000], loss: 0.117772
Epoch[560/1000], loss: 0.116114
Epoch[580/1000], loss: 0.114525
Epoch[600/1000], loss: 0.113000
Epoch[620/1000], loss: 0.111534
Epoch[640/10

In [93]:
# model.eval()
# predict = model(Variable(T_train))
# predict = predict.data.numpy()
# plt.plot(T_train.numpy(), t_train.numpy(), 'ro', label='Original data')
# plt.plot(T_train.numpy(), predict, label='Fitting Line')

# plt.legend() 
# plt.show()

# torch.save(model.state_dict(), './linear.pth')

In [96]:
P_test = X_test.astype(np.float32)
T_test = torch.from_numpy(P_test)

In [97]:
y_test = y_test.values
p_test = y_test.astype(np.float32)
t_test = torch.from_numpy(p_test)

In [99]:
predict = model(Variable(T_test))
predict = predict.data.numpy()

In [101]:
for i in range(len(pred)):
    